In [68]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd, numpy as np, re, time
import itertools
import seaborn as sns
sns.set(style = "whitegrid", color_codes = True,font_scale = 1.5)
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [69]:
from collections import defaultdict
import re

import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout
from keras.models import Sequential
from keras.models import Model

import pickle
import pandas as pd
import numpy as np
# from sklearn.cross_validation import train_test_split
# from Word2VecUtility import Word2VecUtility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from collections import defaultdict



# Reddit Dataset

# Data Cleaning
### Figuring out the "nan" float values that are causing issues

In [70]:
df = pd.read_csv("train-balanced-sarcasm.csv")

In [71]:
print(len(df))

1010826


In [72]:
print(df.isnull().any(axis = 0))

label             False
comment            True
author            False
subreddit         False
score             False
ups               False
downs             False
date              False
created_utc       False
parent_comment    False
dtype: bool


In [73]:
features = df['comment']
labels = df['label']
print(len(features))
print(len(labels))

1010826
1010826


In [74]:
start = time.time()

counter = 0
i_count = 0
bad_indices = []

for i,string in enumerate(features):
    try:
        if isinstance(string, float):
            counter += 1
            bad_indices.append(i_count)
    except:
        pass
    i_count += 1

print(counter)
print("i_count:", i_count)
print("bad_indices:", bad_indices)

end = time.time()
time_elapsed = end-start
print(time_elapsed, "seconds")

53
i_count: 1010826
bad_indices: [56269, 68590, 135348, 199910, 258718, 284331, 312969, 328775, 331735, 332600, 332631, 362293, 389792, 445204, 505371, 520619, 524263, 529336, 532823, 569280, 645450, 651242, 661519, 675235, 683899, 747602, 799033, 800812, 813274, 817886, 859333, 875251, 878050, 898863, 905291, 914178, 914615, 918700, 919882, 923678, 936221, 949593, 966886, 967116, 978220, 982492, 992907, 995023, 1001185, 1001891, 1002133, 1009303, 1010599]
0.46416211128234863 seconds


In [75]:
print(len(df))

1010826


In [76]:
for b in bad_indices:
    df = df.drop([b])
print(len(df))

1010773


In [77]:
print(len(df))

1010773


In [78]:
features = df['comment']
labels = df['label']
print(len(features))
print(len(labels))

1010773
1010773


In [79]:
start = time.time()

counter = 0
i_count = 0
for i in df.itertuples():
    if not i[2]:
        counter += 1
        print(counter)
    i_count += 1

print(counter)
print("i_count", i_count)

end = time.time()
time_elapsed = end-start
print(time_elapsed, "seconds")

0
i_count 1010773
1.845754861831665 seconds


### Select desired sample size:

In [85]:
sample_size = input()

10000


In [86]:
df = df.sample(int(sample_size))
df.reset_index(drop=True,inplace=True)

In [87]:
features = df['comment']
labels = df['label']
print(len(features))
print(len(labels))

10000
10000


# CNN Reddit

In [88]:
import pickle
from sklearn.model_selection import train_test_split
# from Word2VecUtility import Word2VecUtility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from collections import defaultdict

def get_volcabulary_and_list_words(data):
    reviews_words = []
    volcabulary = defaultdict(int)
    for review in data["comment"]:
        # review_words = Word2VecUtility.review_to_wordlist(review, remove_stopwords=True)
        review_words = review.split()
        reviews_words.append(review_words)
        for word in review_words:
            volcabulary[word] += 1
    return volcabulary, reviews_words

def get_reviews_word_index(reviews_words, volcabulary, max_words, max_length):
    volcabulary = sorted(volcabulary.items(), key = lambda x : x[1], reverse = True)[:max_words]
    word2index = {word[0]: i for i, word in enumerate(volcabulary)}
    reviews_words_index = [[start] + [(word2index[w] + index_from) if w in word2index else oov for w in review] for review in reviews_words]
    # in word2vec embedding, use (i < max_words + index_from) because we need the exact index for each word, in order to map it to its vector. And then its max_words is 5003 instead of 5000.
    # padding with 0, each review has max_length now.
    reviews_words_index = sequence.pad_sequences(reviews_words_index, maxlen=max_length, padding='post', truncating='post')
    return reviews_words_index

data = df

# data processing para
max_words = 5000
max_length = 50

# model training parameters
batch_size = int(0.8*len(data))//2
embedding_dims = 100
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

# index trick parameters
index_from = 3
start = 1
# padding = 0
oov = 2


print('get volcabulary...')
volcabulary, reviews_words = get_volcabulary_and_list_words(data)
print('get reviews_words_index...')
reviews_words_index = get_reviews_word_index(reviews_words, volcabulary, max_words, max_length)

print(reviews_words_index[:20, :12])
print(reviews_words_index.shape)

labels = data["label"]
labels[labels == 0] = 0
labels[labels == 1] = 1

pickle.dump((reviews_words_index, labels), open("399850by50reviews_words_index.pkl", 'wb'))
# (reviews_words_index, labels) = pickle.load(open("399850by50reviews_word2vec_words_index.pkl", 'rb'))


index = np.arange(reviews_words_index.shape[0])
train_index, valid_index = train_test_split(
    index, train_size=0.8, random_state=520)

train_data = reviews_words_index[train_index]
valid_data = reviews_words_index[valid_index]
train_labels = labels[train_index]
valid_labels = labels[valid_index]
print(train_data.shape)
print(valid_data.shape)

del(labels, train_index, valid_index)

print("start training model...")

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_words + index_from, embedding_dims, \
                    input_length=max_length))
model.add(Dropout(0.25))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:

# filter_length is like filter size, subsample_length is like step in 2D CNN.
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use standard max pooling (halving the output of the previous layer):
model.add(MaxPooling1D(pool_length=2))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop')
model.fit(train_data, train_labels, batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(valid_data, valid_labels))
scores = model.evaluate(valid_data, valid_labels, verbose=0)
print(scores)

get volcabulary...
get reviews_words_index...
[[   1  797   32    3 3780    2 1026    8    2   19    2 1292]
 [   1  353   11 3782  696    3 2122   69    2    0    0    0]
 [   1  135   32    2    9   45   97 3783   35   11   77   72]
 [   1   95   10   36   38    4  536   38    4  536    0    0]
 [   1   63 2701    8 2702    8 2123   65  697 2703   25   83]
 [   1 1027    3  333   36  266   10   20    4 3785  211    2]
 [   1   71   33  698    8 1149    2    0    0    0    0    0]
 [   1 2704    3  699    9    3 2125  211    3  273   12  537]
 [   1  150   33    3  300 2126  133  354  437    9    2    6]
 [   1  575  109  538   35   15  110   54 2123   32    2 3790]
 [   1  438   10   85   13   99 3792   55    2    2    0    0]
 [   1  423   41  229   29 2705    2    0    0    0    0    0]
 [   1  111    4 1150  240  439   70   52   40  166    2    0]
 [   1  643  935   21    8  133   23  440    0    0    0    0]
 [   1  353  162    3 1029 1151    0    0    0    0    0    0]
 [   1  2

# "News Headline" Dataset

In [51]:
# SOURCE: https://github.com/rishabhmisra/News-Headlines-Dataset-For-Sarcasm-Detection
# NOTES - Dataset is much cleaner, less spelling errors, higher modeling scores overall
df = pd.read_json("Sarcasm_Headlines_Dataset.json", lines = True)

In [52]:
print(len(df))

28619


In [53]:
counter = 0
for i in df.itertuples():
    if i[3] == 1:
        counter += 1
print(counter)

13634


## Use this to sample the dataset

In [54]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,1
1,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails why congress is falling...,0
2,https://www.huffingtonpost.com/entry/eat-your-...,eat your veggies: 9 deliciously different recipes,0
3,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar from getting t...,1
4,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close to using word 'strea...,1


In [55]:
df.iloc[[i+10200 for i in range(9)]]

,article_link,headline,is_sarcastic
10200,https://www.huffingtonpost.com/entry/this-is-u...,'this is us' is finally going to tell us how j...,0
10201,https://www.huffingtonpost.com/entry/the-nfl-s...,the nfl should provide an exemption for medica...,0
10202,https://www.theonion.com/hes-a-stockbroker-say...,"'he's a stockbroker,' says woman who finds tha...",1
10203,https://www.huffingtonpost.com/entry/mass-surv...,democrats and republicans agree more than you'...,0
10204,https://www.huffingtonpost.com/entry/dwight-ho...,dwight howard on helping to empower and educat...,0
10205,https://entertainment.theonion.com/1930s-comed...,1930s comedian pretty sure he's outsmarted mur...,1
10206,https://www.huffingtonpost.com/entry/robin-wri...,robin wright explains why she fought for equal...,0
10207,https://www.theonion.com/horrible-facebook-alg...,horrible facebook algorithm accident results i...,1
10208,https://local.theonion.com/10-pound-fetus-abou...,10-pound fetus about to fucking wreck small mom,1


In [56]:
x = 10207
print(df.iloc[x].loc['is_sarcastic'])
print(df.iloc[x].loc['headline'])

1
horrible facebook algorithm accident results in exposure to new ideas


## EDA

In [57]:
print("Number of Headlines:", len(df))

Number of Headlines: 28619


In [58]:
print(df.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


In [59]:
features = df['headline']
labels = df['is_sarcastic']
print(len(features))
print(len(labels))

28619
28619


# CNN New Headlines

In [60]:
import pickle
from sklearn.model_selection import train_test_split
# from Word2VecUtility import Word2VecUtility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from collections import defaultdict

def get_volcabulary_and_list_words(data):
    reviews_words = []
    volcabulary = defaultdict(int)
    for review in data["headline"]:
        # review_words = Word2VecUtility.review_to_wordlist(review, remove_stopwords=True)
        review_words = review.split()
        reviews_words.append(review_words)
        for word in review_words:
            volcabulary[word] += 1
    return volcabulary, reviews_words

def get_reviews_word_index(reviews_words, volcabulary, max_words, max_length):
    volcabulary = sorted(volcabulary.items(), key = lambda x : x[1], reverse = True)[:max_words]
    word2index = {word[0]: i for i, word in enumerate(volcabulary)}
    reviews_words_index = [[start] + [(word2index[w] + index_from) if w in word2index else oov for w in review] for review in reviews_words]
    # in word2vec embedding, use (i < max_words + index_from) because we need the exact index for each word, in order to map it to its vector. And then its max_words is 5003 instead of 5000.
    # padding with 0, each review has max_length now.
    reviews_words_index = sequence.pad_sequences(reviews_words_index, maxlen=max_length, padding='post', truncating='post')
    return reviews_words_index

# data processing para
max_words = 5000
max_length = 50

# model training parameters
batch_size = int(0.8*len(data))//2
embedding_dims = 100
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

# index trick parameters
index_from = 3
start = 1
# padding = 0
oov = 2


data = pd.read_json(
    'Sarcasm_Headlines_Dataset.json',lines=True)
print('get volcabulary...')
volcabulary, reviews_words = get_volcabulary_and_list_words(data)
print('get reviews_words_index...')
reviews_words_index = get_reviews_word_index(reviews_words, volcabulary, max_words, max_length)

print(reviews_words_index[:20, :12])
print(reviews_words_index.shape)

labels = data["is_sarcastic"]
labels[labels == 0] = 0
labels[labels == 1] = 1

pickle.dump((reviews_words_index, labels), open("399850by50reviews_words_index.pkl", 'wb'))
# (reviews_words_index, labels) = pickle.load(open("399850by50reviews_word2vec_words_index.pkl", 'rb'))


index = np.arange(reviews_words_index.shape[0])
train_index, valid_index = train_test_split(
    index, train_size=0.8, random_state=520)

train_data = reviews_words_index[train_index]
valid_data = reviews_words_index[valid_index]
train_labels = labels[train_index]
valid_labels = labels[valid_index]
print(train_data.shape)
print(valid_data.shape)

del(labels, train_index, valid_index)

print("start training model...")

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_words + index_from, embedding_dims, \
                    input_length=max_length))
model.add(Dropout(0.25))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:

# filter_length is like filter size, subsample_length is like step in 2D CNN.
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use standard max pooling (halving the output of the previous layer):
model.add(MaxPooling1D(pool_length=2))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop')
model.fit(train_data, train_labels, batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(valid_data, valid_labels))
scores = model.evaluate(valid_data, valid_labels, verbose=0)
print(scores)
test_predicted_labels = model.predict(features_test)
print(classification_report(labels_test, test_predicted_labels))

get volcabulary...
get reviews_words_index...
[[   1    2  328 2966    2 2470    4  673 1062    0    0    0]
 [   1    2 2618  703 2967   46  223   12 1854 1257    9    2]
 [   1  898   35    2  660    2  592 1480    0    0    0    0]
 [   1    2 1647    2    2   15  127    3  150    0    0    0]
 [   1  468  452  289 1030    3  520  624    2    2    0    0]
 [   1   77   68    2    0    0    0    0    0    0    0    0]
 [   1  106  254    3 3990   35 2772   11  562 1576    0    0]
 [   1 1710    2    2 2968 1031   24  220   24 1522    4 1367]
 [   1 2773  285  127  136 1855    3  412    6    2 1711  345]
 [   1 2774    4  151  115   22    2    0    0    0    0    0]
 [   1   22 2471   12 3179    8  445    6 3993 1577   62  999]
 [   1    2    2  593   58  532  238   12 1258  625    2    0]
 [   1   36    3  115    2 1578    2    7 1133    0    0    0]
 [   1  521 1784  874  140   86   70    3 1712    7    2    2]
 [   1 1481 3994   13    2   26 1424    2    9    2    0    0]
 [   1 13